`tf.reset_default_graph()` can avoid `重启kernel`<br>
we had better close session to release resource

In [1]:
import tensorflow as tf

## 【迁移学习】往一个已经保存好的 模型添加新的变量
在迁移学习中，通常我们已经训练好一个模型，现在需要修改模型的部分结构，用于我们的新任务。

比如：

在一个图片分类任务中，我们使用别人训练好的网络来提取特征，但是我们的分类数目和原模型不同，这样我们只能取到 fc 层，后面的分类层需要重新写。这样我们就需要添加新的变量。那么这些新加入的变量必须得初始化才能使用。可是我们又不能使用 'tf.global_variables_initializer()' 来初始化，否则原本训练好的模型就没用了。

关于怎么样单独初始化新增的变量，可以参考下面两个链接：
- [1]https://stackoverflow.com/questions/35164529/in-tensorflow-is-there-any-way-to-just-initialize-uninitialised-variables
- [2]https://stackoverflow.com/questions/35013080/tensorflow-how-to-get-all-variables-from-rnn-cell-basiclstm-rnn-cell-multirnn

简单的例子可以直接看下面我的实现方式。

### 初始模型定义与保存
首先定义一个模型，里边有 v1, v2 两个变量，我们把这个模型保存起来。

In [2]:
tf.reset_default_graph()

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

# Create some variables.
v1 = tf.Variable([1.0, 2.3], name="v1")
v2 = tf.Variable(55.5, name="v2")


init_op = tf.global_variables_initializer()
saver = tf.train.Saver()
ckpt_path = '../ckpt/test-model.ckpt'
sess.run(init_op)
save_path = saver.save(sess, ckpt_path, global_step=1)
sess.close()
print("Model saved in file: %s" % save_path)

Model saved in file: ../ckpt/test-model.ckpt-1


**restart(重启kernel然后执行下面cell的代码)**
### 导入已经保存好的模型，并添加新的变量
现在把之前保存好的模型，我只需要其中的 v1 变量。同时我还要添加新的变量 v3。

In [3]:
tf.reset_default_graph()

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

# Create some variables.
v1 = tf.Variable([11.0, 16.3], name="v1")

# ** 导入训练好的模型
saver = tf.train.Saver()
ckpt_path = '../ckpt/test-model.ckpt'
saver.restore(sess, ckpt_path + '-'+ str(1))
print(sess.run(v1))

# ** 定义新的变量并单独初始化新定义的变量
v3 = tf.Variable(666, name='v3', dtype=tf.int32)
init_new = tf.variables_initializer([v3])
sess.run(init_new)
# 。。。这里就可以进行 fine-tune 了
print(sess.run(v1))
print(sess.run(v3))

# ** 保存新的模型。 
#  注意！注意！注意！ 一定一定一定要重新定义 saver, 这样才能把 v3 添加到 checkpoint 中
saver = tf.train.Saver()
saver.save(sess, ckpt_path, global_step=2)

sess.close()

INFO:tensorflow:Restoring parameters from ../ckpt/test-model.ckpt-1
[1.  2.3]
[1.  2.3]
666


**restart(重启kernel然后执行下面cell的代码)**
### 这样就完成了 fine-tune, 得到了新的模型

In [4]:
tf.reset_default_graph()

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

# Create some variables.
v1 = tf.Variable([11.0, 16.3], name="v1")
v3 = tf.Variable(666, name='v3', dtype=tf.int32)
# Add ops to save and restore all the variables.
saver = tf.train.Saver()

# Later, launch the model, use the saver to restore variables from disk, and
# do some work with the model.
# Restore variables from disk.
ckpt_path = '../ckpt/test-model.ckpt'
saver.restore(sess, ckpt_path + '-'+ str(2))
print("Model restored.")

print(sess.run(v1))
print(sess.run(v3))

sess.close()

INFO:tensorflow:Restoring parameters from ../ckpt/test-model.ckpt-2
Model restored.
[1.  2.3]
666
